# Objectifs

- **Manipulation de textes :** transformations diverses pour en produire différentes représentations.

- **Utilisation d'une librairie dédiée à l'apprentissage automatique** : entraîner et évaluer des modèles de classification automatique des textes préparés.

- **Implémentation d'une méthode de classification** qui ne fait pas appel à l'apprentissage automatique.

# Description de la tâche

Classification multi-classe de tweets en français en trois classes de polarité : positif, négatif, neutre

# (Installation et) importation des outils nécessaires

In [ ]:
import os
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import re
# from pprint import pprint

In [ ]:
# nltk : en général limité quant au traitement du français
import nltk

In [ ]:
# En ligne de commande (sans le signe d'exclamation) ou dans une cellule du notebook (avec le signe d'exclamation)
# !pip install -U spacy
## !python -m spacy download fr
# !python -m spacy download fr_core_news_sm

In [ ]:
# spacy : bonne couverture du français ; conçu spécifiquement pour s'interfacer avec des frameworks de deep learning
import spacy
nlp = spacy.load('fr')

# (A) Récupération et mise en forme des données

Données du défi [DEFT2015](https://deft.limsi.fr/2015/corpus.fr.php?lang=fr) : tweets rédigés en français, portant sur la thématique des changements climatiques. Tweets annotés selon leur polarité, pour la tâche 1 du défi : "Classification des tweets selon leur polarité. Étant donné un tweet, cette tâche consiste à le classer, selon l’opinion/sentiment/émotion qu'il exprime, en positif, négatif, neutre ou mixte (si le tweet contient à la fois un sentiment positif et un sentiment négatif)."

__ATTENTION : Ces jeux de données ont été mis à notre disposition EXCLUSIVEMENT à des fins pédagogiques par les organisateurs du défi DEFT 2015. Leur redistribution est formellement interdite et tout travail écrit (rapport de stage, article, etc.) produit sur la base de ces données devra citer les sources indiquées sur le [site Web de DEFT 2015](https://deft.limsi.fr/2015/corpus.fr.php?lang=fr).__

## 0. Structure des données

Les données en apprentissage automatique sont généralement séparées en trois jeux :

+ **entraînement** : données destinées à l'apprentissage du modèle ;

+ **validation** : données destinées à une évaluation intermédiaire du modèle pour permettre l'ajustement de ses hyperparamètres ;

+ **test** : données destinées EXCLUSIVEMENT à l'évaluation FINALE (à réaliser une fois uniquement !) du modèle choisi finalement. Elles ne doivent sous aucune forme servir à la conception du modèle. Il est donc interdit aussi bien de les examiner que d'évaluer le modèle en cours de développement sur ce jeu de données.

Dans notre cas, deux jeux de données sont fournis :
+ entraînement (appelé *Train*), consistant en 7929 observations ;
+ test (appelé *Test*), contenant 3379 observations, soit environ 43% de la taille du jeu d'entraînement.

Nous garderons cette répartition des données, même si un jeu de test plus petit pourrait probablement convenir (ce qui nous permettrait d'exploiter une partie du jeu de test pour l'entraînement). Mais nous devrons prendre soin de ménager nous-mêmes un jeu de validation. À vous d'évaluer votre modèle FINAL sur les données de test !

Les étiquettes (la vérité terrain) et le texte des tweets sont stockés séparément. Nous les regrouperons à partir de l'identifiant du tweet.

## 1. Étiquettes

In [ ]:
def make_label_file_path(parent_dir, train_or_test):
    return os.path.join(parent_dir,
                        f'{train_or_test.title()}_References',
                        'T1.txt')

In [ ]:
make_label_file_path('data', 'train')

In [ ]:
make_label_file_path('data', 'test')

In [ ]:
def map_label_to_numeric(label):
    return 1 if label == '+' else 0 if label == '=' else -1

In [ ]:
def get_labels(parent_dir, train_or_test):
    label_file = make_label_file_path(parent_dir, train_or_test)
    labels = pd.read_table(label_file, header=None, names=['id', 'polarity'])
    labels['polarity'] = labels['polarity'].apply(map_label_to_numeric)
    labels.set_index('id', inplace=True)
    return labels

In [ ]:
train_labels = get_labels('data', 'train')
test_labels = get_labels('data', 'test')

In [ ]:
train_labels.head()

In [ ]:
train_labels.shape, test_labels.shape

## 2. Textes

In [ ]:
def make_data_dir_path(parent_dir, train_or_test):
    return os.path.join(parent_dir, f'deft2015_{train_or_test.upper()}_twitter_raw')

In [ ]:
def get_tweets(parent_dir, train_or_test):
    if train_or_test == 'test':
        train_or_test += 's' # incohérence dans le nommage des répertoires
    data_dir = make_data_dir_path(parent_dir, train_or_test)
    tweets = dict()

    for file_name in sorted(os.listdir(data_dir)):
        if file_name.endswith(".txt"):
            with open(os.path.join(data_dir, file_name), 'r') as f:
                text = f.read().strip()
            id_tweet = int(os.path.splitext(file_name)[0])
            tweets[id_tweet] = text
    
    return (pd.DataFrame.from_dict(tweets, orient='index')
                        .rename(columns={0: 'text'}))

In [ ]:
parent_dir = 'data/twitter'
train_tweets = get_tweets(parent_dir, 'train')
test_tweets = get_tweets(parent_dir, 'test')

In [ ]:
train_tweets.head()

__*QUESTION : Combien de tweets y a-t-il dans le jeu de données d'entraînement et dans celui de test ?*__

In [ ]:
len(train_tweets), len(test_tweets)

## 3. Jonction des textes et des étiquettes

Il y a plus d'étiquettes que de textes, car des tweets ont pu disparaître entre le moment où ils ont été collectés pour l'annotation de référence et le moment où ils ont été récupérés ultérieurement (voir [ici](https://deft.limsi.fr/2015/evaluation.fr.php?lang=fr)). Nous ferons une jointure interne pour ne retenir que les éléments communs aux deux tableaux.

Cependant, tous les tweets disponibles ont une étiquette :

In [ ]:
len(set(train_tweets.index).intersection(set(train_labels.index))) == len(train_tweets)

In [ ]:
def merge_tweets_and_labels(tweets_df, labels_df):
    return pd.merge(tweets_df, labels_df, how='inner',
                    left_index=True, right_index=True)

In [ ]:
train_tweets = merge_tweets_and_labels(train_tweets, train_labels)
test_tweets = merge_tweets_and_labels(test_tweets, test_labels)

In [ ]:
train_tweets.head()

C'est une bonne idée de sauvegarder régulièrement les data frames, pour pouvoir continuer à travailler dessus plus tard si besoin.

In [ ]:
train_tweets.to_pickle('train.pkl')
test_tweets.to_pickle('test.pkl')

Pour lire les fichiers si besoin :

In [ ]:
# train_tweets = pd.read_pickle('train.pkl')
# test_tweets = pd.read_pickle('test.pkl')

## 4. Exploration des données

### 4.1. Distribution des classes

In [ ]:
class_distribution = (pd.DataFrame.from_dict(Counter(train_tweets.polarity.values),
                                             orient='index')
                                  .rename(columns={0: 'num_examples'}))
class_distribution.index.name = 'class'

In [ ]:
class_distribution

In [ ]:
class_distribution['perc_examples'] = np.around(class_distribution.num_examples /
                                                np.sum(class_distribution.num_examples),
                                                2)

In [ ]:
class_distribution

### 4.2. Exploration du texte

**ATTENTION :** Ne pas regarder les données de test !

In [ ]:
train_tweets['text'].values[:15]

# (B) Représentation des textes

## Descripteurs

**Du texte au numérique:**

- les descripteurs (ou encore variables/features/traits/...) sont des unités textuelles : lemmes, racines ou autres ; prises individuellement ou en séquences (n-grammes), mais sans égard à leur ordre ou relations : c'est l'approche en **"sac de mots"** ;

- les valeurs de ces variables sont numériques : binaires (présence/absence du descripteur dans le texte), numériques discrètes (nombre d'occurrences du descripteur dans le texte), numériques continues (différentes pondérations).

## Texte vs corpus

Traditionnellement : les descripteurs textuels sont calculés sur l'ensemble du corpus. Tous les textes sont représentés par le mêmes ensemble de descripteurs, ce qui fait que la représentation d'un **texte** est un grand **vecteur épars** de taille fixe (taille du vecteur = taille du vocabulaire du corpus).

+ Dans notre cas, un texte est un tweet et le corpus est l'ensemble des tweets. Le vocabulaire du corpus est l'ensemble des types (i.e. tokens pris une seule fois) apparaissant dans au moins un tweet.
 
+ **Attention :** le vocabulaire du corpus d'entraînement est aussi celui utilisé pour la prédiction !

Aujourd'hui (état de l'art) : chaque **mot** est représenté par un **vecteur dense** de valeurs réelles. Le texte est représenté par une aggrégation sous une certaine forme des vecteurs de ses mots constituants.

## 1. Sélection de descripteurs : prétraitements textuels

Objectif : réduire le nombre de descripteurs : réduire à un seul descripteur ceux qui sont équivalents (p. ex. deux mots qui ont été écrits avec et sans accents respectivement) ou qui peuvent être regroupés dans une classe d'équivalence (p. ex. remplacer toutes les instances de date par un mot fictif, p. ex. "DATEEXPR").

Quelques procédés : lemmatisation, racinisation, normalisation/correction orthographique, suppression des accents, mise en minuscules, suppression de la ponctuation, suppression de certains mots (mots dits "vides", autres mots), substitution de certains mots par un autre représentant leur appartenance à une classe, etc.

Dans la pratique certains de ces procédés sont souvent appliqués ensemble ou bien ils peuvent être pris en charge par la boîte à outils dans le workflow d'apprentissage automatique (divers paramètres à spécifier à certaines étapes du processus), qui les applique alors en boîte noire. Cependant tous les outils ne gèrent pas au même niveau l'anglais et les autres langues (dont le français), pour lesquelles toutes les options proposées pour l'anglais ne sont pas toujours disponibles.

D'autre part, le choix d'appliquer ou non un certain procédé doit prendre en compte les besoins du contexte concret (p. ex. une mise en minuscules affecte-t-elle la reconnaissance d'entités nommées, si celle-ci est préconisée ?). Pour notre tâche : comment gérer les hashtags, les URL, les noms d'utilisateur (ou pseudos), etc. ?

Comme entraînement à la manipulation du texte, nous verrons quelques exemples de transformation du texte. Nous produirons plusieurs versions de nos textes, qui pourront servir par la suite à l'étape d'apprentissage du classifieur et de prédiction.

Pour illustrer l'effet des différentes transformations sur le texte, prenons comme exemple ce tweet:

In [ ]:
# Exemple
tw = train_tweets['text'].iloc[100]
tw

In [ ]:
tw_nlp = nlp(tw) # spacy
tw_nlp

In [ ]:
# dir(tw_nlp)

### 1.1. Pas de sélection : mots tels quels

In [ ]:
for token in tw_nlp:
    print(token)

In [ ]:
# dir(token)

### 1.2. Réduction par regroupement/uniformisation

#### 1.2.1. Lemmes

In [ ]:
for token in tw_nlp:
    print(token.lemma_)

**EXERCICE.** Produire une version lemmatisée des tweets et la mettre dans une colonne `lemmas` dans la dataframe. Pour ce faire : créer une fonction qui lemmatise un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`). **Attention**, le traitement de toute la colonne peut prendre un peu de temps.

In [ ]:
def lemmatise_text(text):
    text = nlp(text)
    lemmas = [token.lemma_ for token in text]
    return ' '.join(lemmas)

In [ ]:
lemmatise_text(tw)

In [ ]:
train_tweets['lemmas'] = train_tweets['text'].apply(lemmatise_text)

In [ ]:
train_tweets.head()

In [ ]:
test_tweets['lemmas'] = test_tweets['text'].apply(lemmatise_text)

In [ ]:
test_tweets.shape

In [ ]:
# Sauvegarde
train_tweets.to_pickle('train.pkl')
test_tweets.to_pickle('test.pkl')

#### 1.2.2. Racines

In [ ]:
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import SnowballStemmer # ou: from nltk.stem.snowball import FrenchStemmer

In [ ]:
stemmer = SnowballStemmer('french')
tokenizer = TweetTokenizer()
tokenizer.tokenize(tw)

Les tweets présentent des particularités par rapport à d'autres textes. Des outils conçus spécifiquement pour gérer ce type de texte existent. Par exemple, `nltk` propose un tokeniseur pour les tweets:

In [ ]:
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) 
# strip_handles supprime les @...
# reduce_len réduit les séquences de caractères répétés plus de trois fois à des séquences de taille trois
tokenizer.tokenize(tw)

In [ ]:
for token in tokenizer.tokenize(tw):
    print(stemmer.stem(token))

**EXERCICE.** Produire une version racinisée des tweets et la mettre dans une colonne `stems` dans la dataframe. Pour ce faire : créer une fonction qui racinise un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`).

In [ ]:
def stem_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    stemmer = SnowballStemmer('french')
    stems = [stemmer.stem(token) for token in tokenizer.tokenize(text)]
    return ' '.join(stems)

In [ ]:
stem_text(tw)

In [ ]:
train_tweets['stems'] = train_tweets['text'].apply(stem_text)

In [ ]:
train_tweets.head()

In [ ]:
test_tweets['stems'] = test_tweets['text'].apply(stem_text)

In [ ]:
test_tweets.shape

In [ ]:
# Sauvegarde
train_tweets.to_pickle('train.pkl')
test_tweets.to_pickle('test.pkl')

#### 1.2.3. Étiquettes morphosyntaxiques

In [ ]:
for token in tw_nlp:
    print(f'{token.text}\t{token.pos_}')

**EXERCICE.** Produire une version des tweets où chaque token est remplacé par son étiquette morphosyntaxique et la mettre dans une colonne `pos` dans la dataframe. Pour ce faire : créer une fonction qui remplace les mots par leurs étiquettes dans un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`). Attention, le traitement de toute la colonne peut prendre un peu de temps.

In [ ]:
def replace_words_with_pos_tag(text):
    text = nlp(text)
    return ' '.join([token.pos_ for token in text])

In [ ]:
replace_words_with_pos_tag(tw)

In [ ]:
train_tweets['pos'] = train_tweets['text'].apply(replace_words_with_pos_tag)

In [ ]:
train_tweets.head()

In [ ]:
test_tweets['pos'] = test_tweets['text'].apply(replace_words_with_pos_tag)

In [ ]:
test_tweets.shape

In [ ]:
# Sauvegarde
train_tweets.to_pickle('train.pkl')
test_tweets.to_pickle('test.pkl')

#### 1.2.4. Classe d'appartenance des entités nommées

In [ ]:
train_tweets = pd.read_pickle('train.pkl')
test_tweets = pd.read_pickle('test.pkl')

In [ ]:
train_tweets.head()

Limites : reconnaissance imparfaite. Faire des essais pour appréhender les limites de l'outil. Cela nous permettra, par exemple, de corriger certaines erreurs systématiques de l'outil en intervenant en amont sur le texte pour transformer les éléments qui posent difficulté. Par exemple, dans notre cas, l'outil semble ne pas bien gérer les URL. On peut donc penser à les normaliser avant d'appliquer la reconnaissance d'entités nommées. Par ailleurs, un nettoyage du texte en amont peut aussi aider à une meilleure segmentation en tokens et, par conséquent, possiblement aussi à une meilleure reconnaissance des entités nommées (REN, ou NER en anglais).

In [ ]:
tw_nlp.ents

In [ ]:
tw_nlp.ents[0].text

**EXERCICE.** Produire une version des tweets où chaque entité nommée est remplacée par sa classe et la mettre dans une colonne `entites_nommees` dans la dataframe. Pour ce faire : créer une fonction qui remplace les entités nommées reconnues par leurs classe dans un texte ; appliquer cette fonction à la colonne `text` des deux dataframes (`train` et `test`). Attention, le traitement de toute la colonne peut prendre un peu de temps.

In [ ]:
def ner(text):
    original_text = nlp(text)
    entities = {ent.text: ent.label_ for ent in original_text.ents}
    new_text = ''
    for token in original_text:
        token = token.text
        if token in entities:
            label = entities[token]
            new_text += label + ' '
        else:
            new_text += token + ' '
    return new_text

In [ ]:
ner(tw)

In [ ]:
train_tweets['entites_nommees'] = train_tweets['text'].apply(ner)

In [ ]:
train_tweets.head()

In [ ]:
test_tweets['entites_nommees'] = test_tweets['text'].apply(ner)

In [ ]:
test_tweets.shape

In [ ]:
# Sauvegarde
train_tweets.to_pickle('train.pkl')
test_tweets.to_pickle('test.pkl')

#### 1.2.5. Autres classes

Comme exemple, nous remplacerons les adresses Web par un mot fictif URLEXPR. N'hésitez pas à penser à d'autres classes d'équivalence qui vous semblent pertinentes (dates, prix, etc.) !

**EXERCICE.** Créer une fonction `substitute_url` qui prend en entrée une chaîne de caractères et le mot de remplacement (p. ex. "URLEXPR") et remplace les URL présentes dans la chaîne de caractères par le mot de remplacement donné en argument. Indication : utiliser des expressions régulières (module `re`) ; examiner des exemples de tweets (du corpus 
d'entraînement) pour bien saisir la structure des URL. Appliquer ensuite cette fonction à la colonne `text` des dataframes d'entraînement et de test. Dans les deux cas, stocker le résultat de la transformation dans une nouvelle colonne `sans_url`.

In [ ]:
def substitute_url(text, url_replacement):
    text = re.sub(r'https?:\S+', url_replacement, text) # http://t.co/eFKkE9W0GI
    text = re.sub(r'\bwww\.\S+', url_replacement, text) # www.example.com
    return text

In [ ]:
train_tweets['sans_url'] = train_tweets['text'].apply(substitute_url,
                                                      url_replacement='URLEXPR')

In [ ]:
train_tweets.head()

In [ ]:
test_tweets['sans_url'] = test_tweets['text'].apply(substitute_url,
                                                    url_replacement='URLEXPR')

In [ ]:
train_tweets.shape, test_tweets.shape

In [ ]:
# Sauvegarde
train_tweets.to_pickle('train.pkl')
test_tweets.to_pickle('test.pkl')

### 1.3. Réduction par filtrage : suppression de certains mots

#### 1.3.1. Filtrage des mots par fréquence d'utilisation en langue générale : "mots vides"

Critère vague, la notion de mot vide pouvant varier selon le contexte : mots très fréquents en langue générale ou dans un corpus particulier. Ce filtrage est en général géré lors de la vectorisation du corpus (voir plus bas, pour le calcul des valeurs des descripteurs).

In [ ]:
sw = nltk.corpus.stopwords.words('french')

Quelques améliorations :

In [ ]:
'les' in sw # omission importante

In [ ]:
sw.append('les')

In [ ]:
'les' in sw

#### 1.3.2. Filtrage des mots par contenu expressif : mots qui n'ont pas une polarité attestée

Vous pourrez, comme exercice optionnel, implémenter une méthode basée sur ce type de filtrage (voir avant-dernière section de ce notebook).

## 2. Calcul des valeurs des descripteurs

Avant de procéder aux calculs, nous séparerons un jeu de données de validation à partir des données d'entraînement initiales. Nous mettons les données de test fournies de côté, exclusivement pour une évaluation finale des modèles.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_tweets['text'],train_tweets['polarity'],train_size=0.75, random_state=5)

In [ ]:
X_train.shape, X_valid.shape

In [ ]:
X_test, y_test = test_tweets['text'], test_tweets['polarity']

### 2.1. Binaire : présence/absence

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# CountVectorizer?

Étudier la documentation du constructeur de la classe. Il propose notamment une pondération binaire (les valeurs sont alors 0 ou 1) ou par valeurs entières (décomptes d'occurrence - voir section suivante).

En plus, de nombreuses options de réduction du vocabulaire sont proposées. En voici quelques-unes :

- suppression des accents : `strip_accents` ;

- mise en minuscule : `lowercase` (par défault `True`) ;

- seuillage sur la fréquence documentaire (c.à.d. le nombre de documents dans lesquels le terme apparaît) ; exemple : `max_df=0.7` signifie qu'on ignore les termes qui sont présentes dans plus de 70% des textes du corpus (ce qui équivaut à l'élimination des mots vides propres au corpus) ; `min_df=5` ignore les termes qui apparaissent dans moins de 5 textes du corpus ;

- seuillage du nombre de variables à retenir : `max_features=1000` ne retient que les 1000 termes qui ont les "term frequency" (nombre d'occurrences dans un texte particulier) les plus élevées ;

- suppression de mots vides : `stop_words` (liste par défaut ou fournie) ;

- ordre des n-grammes : `ngram_range=(min_n, max_n)` extrait les n-grammes dont la taille est entre `min_n` et `max_n` (les deux compris).

In [ ]:
bin_count = CountVectorizer(binary=True).fit(X_train)

In [ ]:
bin_count

In [ ]:
X_train_vectorized_bin = bin_count.transform(X_train)
X_train_vectorized_bin

Le corpus de validation et celui de test doivent être également transformés en matrices document-termes. **Très important :** les termes sont ceux décomptés sur le corpus d'entraînement ; les termes présent dans le corpus de validation ou de test mais absents du corpus d'entraînement seront ignorés. **Ne pas réapprendre donc le vectoriseur !!!**

In [ ]:
X_valid_vectorized_bin = bin_count.transform(X_valid)
X_test_vectorized_bin = bin_count.transform(X_test)

In [ ]:
X_valid_vectorized_bin # même nombre de "colonnes" (mots) que X_train_vectorized_bin

### 2.2. Numérique discret : décomptes d'occurrence

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer # déjà importé

Calcul des fréquences d'occurrence (tf) des termes dans le corpus, avec les options par défaut :

In [ ]:
vect_count = CountVectorizer().fit(X_train) # binary=False

In [ ]:
vect_count

Examinons le vocabulaire de notre corpus :

In [ ]:
vect_count.get_feature_names()[:50]

In [ ]:
vect_count.get_feature_names()[-50:]

In [ ]:
len(vect_count.get_feature_names()) # taille du vocabulaire

Création de la matrice document-termes :

In [ ]:
X_train_vectorized_count = vect_count.transform(X_train)
X_train_vectorized_count

Comme plus haut : transformation des corpus de validation et de test en matrices document-termes, **avec le même vectoriseur**.

In [ ]:
X_valid_vectorized_count = vect_count.transform(X_valid)
X_test_vectorized_count = vect_count.transform(X_test)

Cette fois-ci nous allons inclure des bigrammes dans le vocabulaire:

In [ ]:
vect_count_bigrams = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)
X_train_vectorized_count_bigrams = vect_count_bigrams.transform(X_train)
X_valid_vectorized_count_bigrams = vect_count_bigrams.transform(X_valid)
X_test_vectorized_count_bigrams = vect_count_bigrams.transform(X_test)

In [ ]:
len(vect_count_bigrams.get_feature_names())

### 2.3. Numérique continu : TF-IDF (ou autres pondérations)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Limitons le vocabulaire à des termes qui apparaissent dans au moins 5 documents.

In [ ]:
vect_tfidf = TfidfVectorizer(min_df=5).fit(X_train)

La réduction de la taille du vocabulaire est spectaculaire !

In [ ]:
len(vect_count.get_feature_names()), len(vect_tfidf.get_feature_names())

In [ ]:
X_train_vectorized_tfidf = vect_tfidf.transform(X_train)
X_valid_vectorized_tfidf = vect_tfidf.transform(X_valid)
X_test_vectorized_tfidf = vect_tfidf.transform(X_test)

# Classification des textes

Nous entraînerons des modèles de classification appartenant à quelques familles d'algorithmes d'apprentissage automatique. D'autres familles restent à explorer. L'objectif est de comparer non seulement les performances des différentes méthodes entre elles, mais aussi la performance d'une même méthode sur des représentations différentes du texte.

## Workflow

**Schéma général :** apprentissage (> évaluation sur données de validation > apprentissage > évaluation sur données de validation >...) (> évaluation sur données de test) > prédiction

Boîte à outils (algorithmes pré-implémentés) : `scikit-learn`. Interface unifiée pour l'ensemble des algorithmes. **Mise en oeuvre :**

* si besoin de réduire et/ou centrer les données (réduction statistique) :
    - création d'un objet `scaler` de la classe adaptée (réducteur des données) ;
    - entraînement du `scaler` sur les données d'entraînement : méthode `fit` de l'objet réducteur ;
    - réduction des données d'entraînement : méthode `transform` du réducteur ; cette étape peut être enchaînée avec la précédente grâce à la méthode `fit_transform` du réducteur ;
    - réduction des données de validation et de test : méthode `transform` du réducteur (attention : même réducteur que pour les données d'entraînement ! On ne réapprend pas les critères de réduction sur les données de validation/test !) ;


* création de l'objet estimateur : appel du constructeur de la classe pertinente, avec d'éventuels paramètres si valeurs autres que défaut ;

* apprentissage de l'estimateur sur les données d'entraînement (éventuellement réduites) : méthode `fit` de l'estimateur ; cette étape peut être enchaînée avec la précédente ;

* évaluation de l'estimateur sur les données d'entraînement, de validation et/ou (uniquement si c'est le modèle final !) de test : méthode `score` de l'estimateur.

* prédiction sur des données nouvelles : méthode `predict` de l'estimateur.

**Métriques d'évaluation :**

* taux de bonne classification (*accuracy*) ;

* précision (*precision*) ;

* rappel (*recall*) ;

* score F1 ;

* aire sous la courbe ROC (*ROC AUC*) : pour la classification binaire ;

* métriques "maison", sur mesure.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

### 1. Modèles de référence faibles (*weak baselines*)

#### 1.1. Choix aléatoire

Toutes les classes ont les mêmes chances d'être choisies (prédiction uniforme) ou bien le prédicteur respecte la distribution des classes dans les données d'entraînement.

In [ ]:
from sklearn.dummy import DummyClassifier

Prédiction proportionnelle à la distribution des classes dans les données d'entraînement :

In [ ]:
random_prop_class = DummyClassifier(strategy='stratified').fit(X_train_vectorized_tfidf,
                                                               y_train)
predictions_valid = random_prop_class.predict(X_valid_vectorized_tfidf)
conf_mat = confusion_matrix(y_valid, predictions_valid)

In [ ]:
print(conf_mat)

In [ ]:
accuracy_score(y_valid, predictions_valid)

Prédiction uniforme :

In [ ]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_vectorized_tfidf,
                                                         y_train)
predictions_valid = random_uniform.predict(X_valid_vectorized_tfidf)
predictions_valid

In [ ]:
conf_mat = confusion_matrix(y_valid, predictions_valid)

In [ ]:
print(conf_mat)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

#### 1.2. Prédiction constante de la classe majoritaire

Seules les instances de la classe majoritaire seront classées correctement.

**EXERCICE.** Quel est le taux de bonne classification pour cette approche dans notre scénario ?

In [ ]:
class_distribution

Vérifiez votre réponse :

In [ ]:
maj = DummyClassifier(strategy='most_frequent').fit(X_train_vectorized_tfidf, y_train)
predictions_valid = maj.predict(X_valid_vectorized_tfidf)
predictions_valid

In [ ]:
maj_class = (class_distribution.index[class_distribution.perc_examples ==
                                      np.amax(class_distribution.perc_examples)][0])
maj_class

In [ ]:
np.all(predictions_valid == maj_class)

In [ ]:
maj.score(X_valid_vectorized_tfidf, y_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

### 2. Classifieur naïf bayesien

En général pris également comme baseline.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model_nb = MultinomialNB().fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_valid_vectorized_tfidf)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

### 3. Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression

**EXERCICE.** Entraîner un modèle avec les arguments suivants : `multi_class='multinomial'`, `solver='lbfgs'` sur le corpus vectorisé par nombre d'occurrences et l'évaluer sur le corpus de validation. Si vous recevez un message indiquant que l'algorithme d'optimisation a du mal à converger, augmentez le nombre d'itérations (paramètre `max_iter`, par défaut 100).

In [ ]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                              max_iter=200).fit(X_train_vectorized_count, y_train)

In [ ]:
predictions_valid = model_lr.predict(X_valid_vectorized_count)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

Examinons les variables (termes) ayant l'association la plus forte avec chaque classe.

In [ ]:
def print_n_strongly_associated_features(vectoriser, model, n):
    feature_names = np.array(vectoriser.get_feature_names())

    for i in range(3):
        class_name = model.classes_[i]
        print("CLASSE {}".format(class_name))
        idx_coefs_sorted = model.coef_[i].argsort() # ordre croissant
        print("Les dix variables ayant l'association négative la plus forte " + 
              "avec la classe {} :\n{}\n".format(class_name,
                                                 feature_names[idx_coefs_sorted[:n]]))
        idx_coefs_sorted = idx_coefs_sorted[::-1] # ordre décroissant
        print("Les dix variables ayant l'association positive la plus forte " +
              "avec la classe {} :\n{}\n"
              .format(class_name,
                      feature_names[idx_coefs_sorted[:n]]))
        print()

In [ ]:
print_n_strongly_associated_features(vect_count, model_lr, 10)

**EXERCICE.** Entraîner un modèle avec les arguments suivants : `multi_class='multinomial'`, `solver='lbfgs'` sur le corpus vectorisé par TF-IDF et l'évaluer sur le corpus de validation.

In [ ]:
model_lr = LogisticRegression(multi_class='multinomial',
                              solver='lbfgs').fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_tfidf)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

La performance est légèrement inférieure, mais nous l'avons obtenue en utilisant considérablement moins de variables.

In [ ]:
feature_names = np.array(vect_tfidf.get_feature_names())
idx_tfidf_sorted = X_train_vectorized_tfidf.max(0).toarray()[0].argsort()
print("TF-IDF le moins élevé : {}".format(feature_names[idx_tfidf_sorted[:10]]))
print("TF-IDF le plus élevé : {}".format(feature_names[idx_tfidf_sorted[:-11:-1]]))

Avec le vectoriseur à **unigrammes et bigrammes** :

In [ ]:
model_lr = LogisticRegression(multi_class='multinomial',
                              solver='lbfgs').fit(X_train_vectorized_count_bigrams,
                                                  y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_count_bigrams)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

In [ ]:
print_n_strongly_associated_features(vect_count_bigrams, model_lr, 10)

### 4. SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
model_svm = SVC(kernel='linear', C=0.1).fit(X_train_vectorized_count_bigrams, y_train)
predictions_valid = model_svm.predict(X_valid_vectorized_count_bigrams)

In [ ]:
accuracy_score(y_valid, predictions_valid)

In [ ]:
print(classification_report(y_valid, predictions_valid))

# Exercices optionnels

## 1. Implémentation d'une méthode sans apprentissage automatique

Cette méthode, qui pourrait servir de modèle de référence forte (*strong baseline*) pour notre tâche, utilise  un lexique qui recense des mots et leur polarité. Elle consiste à calculer la polarité globale d'un texte en agrégeant les polarités des mots qu'il contient. Concrètement, dans sa version la plus simplifiée, cette méthode calcule la polarité d'un texte selon la formule : sign(nombre_de_termes_positifs - nombre_de_termes_négatifs), donc -1 si la somme est négative, 0 si elle est nulle et 1 si elle est positive ([Hu et Liu (2004)](https://pdfs.semanticscholar.org/13e5/f0c40c85ca8e01b3756963d5352358de7c29.pdf), [Kim et Hovy (2004)](http://anthology.aclweb.org/P/P06/P06-2.pdf#page=493)). Cela revient à un filtrage des mots, suivi d'un calcul de score. Des variantes affinées de cette approche ont également été proposées.

Implémentez cette approche et testez-la sur nos corpus, en vous servant du lexique [FEEL](http://advanse.lirmm.fr/feel.php) (voir [article](https://hal-lirmm.ccsd.cnrs.fr/lirmm-01348016/document)) : une liste de 14128 **lemmes** annotés en termes de polarité (positif/négatif) et de six émotions (joy, fear, sadness, anger, surprise, disgust). Pour cet exercice seule la colonne polarité du lexique est à retenir. Faites bien attention à ce que les formes lexicales présentes dans le texte et celles du lexique soient comparables !

Vous pouvez aussi, éventuellement, explorer l'attribut `sentiment` des tokens générés par `spacy`.

Réflection : cette approche vous semble-t-elle raisonnable ? Quelle est sa performance si on la compare à celle des méthodes basées sur l'apprentissage automatique que vous avez testées ? Quels en sont, d'après vous, les points forts et les points faibles ? 

## 2. Sélection de modèle

Pour les algorithmes que vous avez testés (ou d'autres, voir les pistes pour approfondissement), essayez de trouver une combinaison des hyper-paramètres et de la représentation des textes qui entraîne une amélioration de la performance. Pouvez-vous atteindre un taux de bonne classification (*accuracy*) d'au moins 70% sur le corpus de validation ?

Une fois votre modèle choisi sur la base de sa performance sur le corpus de validation, évaluez-le sur le corpus de test et rapportez le résultat. Cette évaluation est à faire une seule fois ! Elle est censée donner une estimation aussi fiable que possible du pouvoir de généralisation de votre modèle. Il n'est pas question d'affiner encore le modèle après cette évaluation, puis de l'évaluer à nouveau sur les données de test : à ce moment-là, votre corpus de test sera devenu un simple corpus de validation.

# Pistes pour l'approfondissement

**1.** Pousser la réduction de variables plus loin : correction/normalisation de l'orthographe, autres transformations considérées pertinentes. Il n'y a pas de recette universelle, il faut essayer différentes approches et voir ce qui marche le mieux sur nos textes et notre tâche. Le texte des tweets pose des problèmes particuliers, il faut donc trouver les traitements les mieux adaptés.

**2.** Combiner des descripteurs textuels avec des variables non-textuelles. Il faudra construire explicitement (pas en version sparse matrix) la matrice document-termes (et appliquer probablement un filtrage plus agressif, pour des raisons de coût de mémoire) pour pouvoir l'augmenter d'autres variables. Le fichier `json` correspondant à chaque tweet contient des méta-données. En choisir une ou deux, les extraire et les ajouter à la représentation des données. Puis entraîner et évaluer un classifieur sur ce nouveau jeu de descripteurs.

**3.** Essayer d'autres algorithmes de classification, par exemple : arbres de décision (`from sklearn.tree import DecisionTreeClassifier`), méthodes d'ensemble (forêts aléatoires : `from sklearn.ensemble import RandomForestClassifier` ; gradient-boosted decision trees : `from sklearn.ensemble import GradientBoostingClassifier`), réseaux de neurones simple (`from sklearn.neural_network import MLPClassifier`) ou bien des architectures plus complexes (autres librairies : `tensorflow`, `keras`, etc.).

**4.** Ajuster les hyper-paramètres d'un modèle par validation croisée (`from sklearn.model_selection import GridSearchCV`). Utiliser dans ce cas la totalité du jeu d'entraînement fourni initialement (ne plus en séparer une portion pour la validation).